In [320]:
import pandas as pd
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARMA
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
%matplotlib inline
#%matplotlib notebook
sns.set(rc={'figure.figsize':(11.7,8.27)})
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime,timedelta
from itertools import product
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [8]:
keys = pd.read_excel(r'key_match_2019_16-18_data.xlsx',sheetname='final_keys')['keys'].tolist()
df_16_18 = pd.read_excel(r'dataprep1.xlsx',sheetname='model_data_16_18')
df_2019 = pd.read_excel(r'2019_monthly_data.xlsx',sheetname='model_data_2019')

# df_16_18.to_pickle('df_16_18.pkl')
# df_2019.to_pickle('df_2019.pkl')
# df_16_18.columns = ['Distributor', 'STRIPES_Code', 'month-year', 'Date', 'year','SIV', 'timestamp', 'key', 'if_exist']
# df_2019.columns = ['Distributor', 'Material', 'timestamp', 'YEAR', 'EMAPS_Forecast', 'SIV',
#        'is_demand', 'key', 'if_in_demand']

#key = keys[0]

In [55]:
# ser_16_18 = df_16_18.query('key == @key')[['timestamp','SIV']].set_index('timestamp')
# ser_2019 = df_2019.query('key == @key')[['timestamp','SIV']].set_index('timestamp')
# ser = ser_16_18.append(ser_2019).sort_index()

In [60]:
# base_time_frame = pd.date_range(start="2016-01-01",end="2020-02-01",freq="MS")
# key_list = [key]*len(base_time_frame)

In [68]:
# # tm_key = [[tm,key] for tm in base_time_frame]
# # tm_key_df = pd.DataFrame(tm_key,columns=['date','matCode'])

# # tm_key_ser = pd.Series(key_list,index=base_time_frame)
# # siv_df = pd.concat([tm_key_ser,ser],axis=1)
# # siv_df.columns = ['key','demand']

# tm_df = pd.DataFrame(index=base_time_frame)
# tm_df = pd.concat([tm_df,ser],axis=1)

# #sample_data = siv_df.demand.interpolate(method='linear')
# #sample_data = siv_df.demand.interpolate(method='spline',order=2)
# # sample_data.plot()
# # siv_df.demand.interpolate(method='spline',order=2).plot()
# sample_data = tm_df.interpolate(method='time')
# #sample_data.plot()

In [ ]:
# tm_df = pd.DataFrame(index=base_time_frame)
# tm_df = pd.concat([tm_df,ser],axis=1)
# sample_data = tm_df.interpolate(method='time')

In [ ]:
# def get_forecast(data,tm_df,tm,key):
    
#     train_data = data.loc[:tm][:-1]
#     demand_act = tm_df.loc[tm].values[0] 
#     try:
#         model = ARIMA(train_data, order=(4,1,1))
#         model_fit = model.fit(transparams=False,disp=-1,trend='c',start_ar_lags =5,maxiter = 5000)
#         fcst = model_fit.forecast(steps=1)[0]
#     except Exception as e:
#         print(e)
#         fcst = 0
# return([key,tm.strftime("%Y-%m-%d"),demand_act,fcst])

# for key in keys:
    
#     ser_16_18 = df_16_18.query('key == @key')[['timestamp','SIV']].set_index('timestamp')
#     ser_2019 = df_2019.query('key == @key')[['timestamp','SIV']].set_index('timestamp')
#     ser = ser_16_18.append(ser_2019).sort_index()
#     tm_df = pd.DataFrame(index=base_time_frame)
#     tm_df = pd.concat([tm_df,ser],axis=1)
#     sample_data = tm_df.interpolate(method='time')
#     tm_df.fillna(0,inplace=True)
    
#     fcsts.append(list(map(lambda x: get_forecast(sample_data,tm_df,x,key),forecast_range)))   
    
# res_df = pd.DataFrame(res,columns=['key','Date','actual','forecast'])

In [123]:
# fixed params
base_time_frame = pd.date_range(start="2016-01-01",end="2020-02-01",freq="MS")
train_data_max_date = datetime.strptime('20190701',"%Y%m%d")
forecast_range = list(filter(lambda x: x > train_data_max_date,base_time_frame))
res = list()

In [353]:
res = list()
for key in keys:
    ser_16_18 = df_16_18.query('key == @key')[['timestamp','SIV']].set_index('timestamp')
    ser_2019 = df_2019.query('key == @key')[['timestamp','SIV']].set_index('timestamp')
    ser = ser_16_18.append(ser_2019).sort_index()
    #ser = ser[ser<=ser.quantile(0.99)]
    ser = ser[ser < ser.quantile(0.75)+ (ser.quantile(0.75)-ser.quantile(0.25))*2]
    tm_df = pd.DataFrame(index=base_time_frame)
    tm_df = pd.concat([tm_df,ser],axis=1)
    sample_data = tm_df.interpolate(method='time')
    tm_df.fillna(0,inplace=True)
    print(key)
    
    for tm in forecast_range:
        train_data = sample_data.loc[:tm][:-1]
        demand_act = tm_df.loc[tm].values[0]
        naive_fcst = train_data.iloc[-1].values[0]
        avg_fcst = train_data.mean()
        sma2 = train_data.rolling(5).mean().iloc[-1][0]
        sma5 = train_data.rolling(5).mean().iloc[-1][0]
        fit2 = SimpleExpSmoothing(np.asarray(train_data)).fit(smoothing_level=0.6,optimized=False)
        ses = fit2.forecast(1)[0]
        fit1 = Holt(np.asarray(train_data)).fit(smoothing_level = 0.3,smoothing_slope = 0.1)
        holt = fit1.forecast(1)[0]
        hwm_model = ExponentialSmoothing(np.asarray(train_data) ,seasonal_periods=7 ,trend='add', seasonal
                ='add',).fit()
        hwm = hwm_model.forecast(1)[0]
        #print(tm.date())
        fcst = 0
        stderr = 0
        up_conf = 0
        low_conf = 0
        try:
            model = ARIMA(train_data, order=(2,1,1))
            model_fit = model.fit(transparams=False,disp=-1,trend='c',maxiter = 5000)#start_params =5
            #print(model_fit.forecast())
            fcst,stderr,conf = model_fit.forecast()
            fcst = 0 if np.isnan(round(fcst[0])) else round(fcst[0])
            stderr = 0 if np.isnan(round(stderr[0])) else round(stderr[0])
            low_conf = 0 if np.isnan(round(conf[0][0])) else round(conf[0][0])
            up_conf = 0 if np.isnan(round(conf[0][1])) else round(conf[0][1])
            #print(fcst,stderr)
        except Exception as e:
            print(e)
            fcst = 0
            fcst = 0
            stderr = 0
            up_conf = 0
            low_conf = 0
        
        res.append([key,tm,demand_act,fcst,stderr,low_conf,up_conf,naive_fcst,\
                    avg_fcst,sma2,sma5,ses,holt,hwm]) #tm.strftime("%Y-%m-%d")
res_df = pd.DataFrame(res,columns=['key','timestamp','actual','forecast','stdErr','lowPrInt','upPrInt','naive',\
                                  'avg_fcst','sma2','sma5','ses','holt','hwm'])

Super Transports P Ltd-139542
Super Transports P Ltd-139530
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.
Super Transports P Ltd-122310
Super Transports P Ltd-139546
SVD did not converge
Super Transports P Ltd-139306
Super Transports P Ltd-139434
Super Transports P Ltd-139495
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
Super Transports P Ltd-122330
Super Transports P Ltd-139461
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
exog contains inf or nans
Super Transports P Ltd-13

In [354]:
res_df_2 = pd.merge(res_df,df_2019[['key','EMAPS_Forecast','timestamp']],how='left',on=['key','timestamp'])
res_df_2['fcst2'] = res_df_2['forecast']
res_df_2.loc[(res_df_2.fcst2==0),'fcst2'] = res_df_2.naive

In [193]:
# from sklearn.metrics import mean_squared_error
# mse1 = mean_sqaured_error(res_df_2.actual,res_df_2.forecast)
# mse2 = mean_sqaured_error(res_df_2.actual,res_df_2.EMAPS_Forecast)

ImportError: cannot import name 'comb'

In [296]:
# mae1 = (res_df_2.actual-res_df_2.forecast).abs().mean()
# mae2 = (res_df_2.actual-res_df_2.EMAPS_Forecast).abs().mean()
# mae3 = (res_df_2.actual-res_df_2.naive).abs().mean()
# mae4 = (res_df_2.actual-res_df_2.fcst2).abs().mean()

# print(mae1,mae2,mae3,mae4)

142465552233.72556 2223.8335888767715 2807.08716082223 142465552193.40085


In [255]:
# res_df_2.to_excel('res.xlsx',index=False)

In [ ]:
# ser = ser[ser<=ser.quantile(0.99)]

# ser[ser < ser.quantile(0.75)+ (ser.quantile(0.75)-ser.quantile(0.25))*2]

#res_df_2.forecast.quantile([0.01,0.90])

In [355]:
res_df_3 = res_df_2.query('0<= forecast <= 6267')

res_df_3 = res_df_3.loc[res_df_3.actual != 0]

In [356]:
mae1 = (res_df_3.actual-res_df_3.forecast).abs().mean()
mae2 = (res_df_3.actual-res_df_3.EMAPS_Forecast).abs().mean()
mae3 = (res_df_3.actual-res_df_3.naive).abs().mean()
mae4 = (res_df_3.actual-res_df_3.fcst2).abs().mean()

mae5 = (res_df_3.actual-res_df_3.avg_fcst).abs().mean()
mae6 = (res_df_3.actual-res_df_3.sma2).abs().mean()
mae7 = (res_df_3.actual-res_df_3.sma5).abs().mean()
mae8 = (res_df_3.actual-res_df_3.ses).abs().mean()
mae9 = (res_df_3.actual-res_df_3.holt).abs().mean()
mae10 = (res_df_3.actual-res_df_3.hwm).abs().mean()

print(mae1,mae2,mae3,mae4,mae5,mae6,mae7,mae8,mae9,mae10)

2203.525506828528 1857.4497738998484 1983.0589683801356 1793.8420879602247 1572.8081319160462 1491.2038869072012 1491.2038869072012 1155.38173712664 1078.4755540510987 1114.5804065690204


In [ ]:
# still to try SARIMAX but need hyper parameter tuning for the models will take time 